In [1]:
# TODO DELETE ME
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# TODO

# find accuracy of Base model (using Spacy Root Verb Dependency)
# find accuracy using vanilla CRF Suite
# explore features and hyperparameters of CRF Suite Algo
# find accuracy using tweaked CRF Suite
# display results using Displacy
# interpret results using eli5

<IPython.core.display.Javascript object>

In [3]:
import warnings

warnings.filterwarnings("ignore")

<IPython.core.display.Javascript object>

In [30]:
import pandas as pd
import math

# to set random seed
import numpy as np

# used to create CRF model
from sklearn_crfsuite import CRF

# used to evaluate model
from sklearn_crfsuite import metrics

# for hyperparameter tuning parameter range
import scipy

# for specifying f1 metrics
from sklearn.metrics import make_scorer

# for cross validation of hyperparameters
from sklearn.model_selection import RandomizedSearchCV

# to visualize the weight of parameters of the fitted model
import eli5

<IPython.core.display.Javascript object>

In [69]:
pd.set_option("max_row", 100000)

<IPython.core.display.Javascript object>

In [70]:
np.random.seed(42)

<IPython.core.display.Javascript object>

In [71]:
data = pd.read_csv("tagged_data.csv", index_col=0)

<IPython.core.display.Javascript object>

In [72]:
# Jump to
data.shape
data.head()

,sentence#,word,pos,tag
0,0.0,Preheat,VB,U-Action
1,0.0,oven,NN,U-Utensil
2,0.0,to,IN,O
3,0.0,425,CD,O
4,0.0,degrees,NNS,O


<IPython.core.display.Javascript object>

In [248]:
combined_in = (
    data.groupby(["sentence#"])["word"]
    .apply(lambda x: " ".join(x.astype(str)))
    .reset_index()
)
pd.set_option("max_colwidth", 200)
combined_in.head()

,sentence#,word
0,0.0,Preheat oven to 425 degrees F.
1,1.0,Press dough into the bottom and sides of a 12 inch pizza pan .
2,2.0,Bake for 5 minutes until set but not browned .
3,3.0,Cut sausage into small pieces .
4,4.0,Whisk eggs and milk in a bowl until frothy .


<IPython.core.display.Javascript object>

In [260]:
combined_in_new = combined_in.reset_index()[["sentence#", "word"]].values.tolist()
for item in combined_in_new:
    lower = item[1].lower()
    doc_new = nlp(lower)
    for token in doc_new:
        dd = token.lemma_
        print(dd)

preheat
oven
to
425
degree
f.
press
dough
into
the
bottom
and
side
of
a
12
inch
pizza
pan
.
bake
for
5
minute
until
set
but
not
brown
.
cut
sausage
into
small
piece
.
whisk
egg
and
milk
in
a
bowl
until
frothy
.
spoon
sausage
over
baked
crust
and
sprinkle
with
cheese
.
pour
egg
mixture
slowly
over
sausage
and
cheese
.
s
&
p
to
taste
.
bake
15
-
20
minute
or
until
egg
be
set
and
crust
be
brown
.
brown
ground
beef
in
large
pot
.
add
chop
onion
to
ground
beef
when
almost
brown
and
sautee
until
wilt
.
add
all
other
ingredient
.
add
kidney
bean
if
-PRON-
like
bean
in
-PRON-
chili
.
cook
in
slow
cooker
on
high
for
2
-
3
hour
or
6
-
8
hour
on
low
.
serve
with
cold
clean
lettuce
and
shred
cheese
(
-PRON-
be
a
cajun
thing
)
.
place
potato
in
a
large
pot
of
lightly
salt
water
and
bring
to
a
gentle
boil
.
cook
until
potato
be
just
tender
.
garnish
with
finely
dice
red
and
yellow
bell
pepper
.
drain
.
place
potato
in
a
large
bowl
and
add
all
ingredient
except
the"alouette
"
.
mix
well
and
transfer


drain
,
reserve
1
1/4
cup
cooking
liquid
;
in
a
food
processor
,
process
the
potato
and
spinach
until
very
smooth
,
add
the
butter
1/2
t
at
a
time
.
return
the
puree
to
the
pot
and
add
the
reserved
liquid
until
the
desire
consistency
;
reheat
slowly
and
season
with
salt
and
pepper
;
ladle
into
shallow
soup
plate
and
garnish
with
nutmeg
or
shave
cheese
.
in
a
3
qt
saucepan
over
medium
high
heat
,
saute
the
onion
and
scallion
in
the
apple
juice
and
oil
for
5
minute
,
stir
frequently
.
add
the
spinach
,
parsley
and
celery
.
cook
for
5
-
7
minute
,
stir
occasionally
add
the
broth
,
oats
,
salt
,
thyme
and
pepper
.
bring
to
a
boil
,
then
lower
the
heat
to
medium
.
cover
and
simmer
for
20
minute
.
remove
from
the
heat
.
let
the
soup
cool
for
10
minute
.
work
in
batch
,
puree
in
a
blender
until
thick
and
smooth
.
return
to
the
pot
.
reheat
and
serve
.
combine
soup
,
egg
and
seasoned
salt
in
a
bowl
and
set
aside
(
soup
be
already
salt
so
be
careful
on
amount
of
season
salt
-PRON-
add
)
.
mix
t

<IPython.core.display.Javascript object>

In [261]:
dd

'!'

<IPython.core.display.Javascript object>

In [73]:
#################################################
# Dependency parser
###################################################

# Import modules
import spacy
from spacy import displacy

nlp = spacy.load("en_core_web_sm")

<IPython.core.display.Javascript object>

In [74]:
# Locate validation set from sentence#212 onward (ID:212 to ID:263)
parse_data = data.iloc[2770:3445]

# Inspect the dataset
parse_data.head()

,sentence#,word,pos,tag
2770,212.0,Spray,NN,U-Action
2771,212.0,baking,NN,B-Utensil
2772,212.0,sheet,NN,L-Utensil
2773,212.0,with,IN,O
2774,212.0,non,JJ,B-Utensil


<IPython.core.display.Javascript object>

In [75]:
# Set the sentenceID as index for merging ID:212 to ID:263
parse_data.set_index("sentence#", inplace=True)
parse_data.head()

,word,pos,tag
sentence#,,,
212.0,Spray,NN,U-Action
212.0,baking,NN,B-Utensil
212.0,sheet,NN,L-Utensil
212.0,with,IN,O
212.0,non,JJ,B-Utensil


<IPython.core.display.Javascript object>

In [76]:
# Merge the words back to its orginal form by sentence ID
combined_sentence = (
    parse_data.groupby(["sentence#"])["word"]
    .apply(lambda x: " ".join(x.astype(str)))
    .reset_index()
)
pd.set_option("max_colwidth", 200)
combined_sentence.head()

,sentence#,word
0,212.0,Spray baking sheet with non - stick spray .
1,213.0,Spread potatoes onto baking sheet .
2,214.0,Bake for 15 minutes ; toss and turn ; bake for five minutes ; add herbs and toss ; bake a final five minutes until the potatoes are brown and crisp .
3,215.0,Serve immediately .
4,216.0,"In 1/4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes ."


<IPython.core.display.Javascript object>

In [77]:
# Convert the dataframe to the nested list
combined_sentences_list = combined_sentence.reset_index()[
    ["sentence#", "word"]
].values.tolist()
for item in combined_sentences_list:
    print(item[1])

Spray baking sheet with non - stick spray .
Spread potatoes onto baking sheet .
Bake for 15 minutes ; toss and turn ; bake for five minutes ; add herbs and toss ; bake a final five minutes until the potatoes are brown and crisp .
Serve immediately .
In 1/4 cup butter , saute carrots , onion , celery and broccoli stems for 5 minutes .
Add thyme , oregano and basil ; saute 5 minutes more .
Add wine and deglaze pan .
Add hot chicken stock and reduce by one - third .
Add Worcestershire sauce , Tabasco , smoked chicken , beans and broccoli florets ; simmer 5 minutes .
Add cream , simmer 5 minutes more and season to taste ( thicken with cornstarch if desired ) .
Drop in remaining butter , piece by piece , stirring until melted and serve immediately .
Smoked Chicken : On a covered grill , slightly smoke boneless chicken , cooking to medium rare ( about 30 minutes ) .
Chef Meskan uses applewood chips and does not allow the grill to become too hot .
Preheat oven to 350 degrees .
Grease 13 x 9 b

<IPython.core.display.Javascript object>

In [78]:
# Print(combined_sentence_list)
dp_picked_sent = []
not_dp_picked_sent = []

for sentence in combined_sentences_list:
    ind_sentences = sentence[1]
    doc = nlp(ind_sentences)
    for token in doc:
        if token.pos_ == "VERB":
            # print(int(sentence[0]), " ".join([token.dep_ for token in doc]))
            dep_parser = " ".join([token.dep_ for token in doc])
            dp_picked_sent.append((int(sentence[0]), dep_parser))

        else:
            non_dep_picked_sent = []
        # dp_picked_sent.append((int(sentence[0]), dep_parser))
        # if token.dep_ == "ROOT" or token.dep_ == "conj" or token.dep_ == "nmod" or token.dep_ == "advcl":
        # print(doc, "---->>>")

<IPython.core.display.Javascript object>

In [82]:
# Print(combined_sentence_list)
dp_picked_sent = []

for sentence in combined_sentences_list:
    ind_sentences = sentence[1]
    doc = nlp(ind_sentences)
    for token in doc:
        dep_parser = " ".join([token.dep_ for token in doc])
        dp_picked_sent.append((int(sentence[0]), dep_parser))
        #############################
        # Code for locate the ACTION words
        #############################
        # if token.pos_ == "VERB":
        # if token.dep_ == "ROOT" or token.dep_ == "conj":
        # print(doc, "---->>>", token.text, token.pos_, token.dep_)
        #############################
        # Code End
        #############################

        # if token.pos_ == "VERB":
        # print(int(sentence[0]), " ".join([token.dep_ for token in doc]))
        # dep_parser = " ".join([token.dep_ for token in doc])
    # dp_picked_sent.append((int(sentence[0]), dep_parser))

    # dp_picked_sent.append((int(sentence[0]), dep_parser))
    # if token.dep_ == "ROOT" or token.dep_ == "conj" or token.dep_ == "nmod" or token.dep_ == "advcl":
    # print(doc, "---->>>")

<IPython.core.display.Javascript object>

In [86]:
# Check the sentence with its ID
dp_picked_sent

[(212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (213, 'compound ROOT prep compound pobj punct'),
 (213, 'compound ROOT prep compound pobj punct'),
 (213, 'compound ROOT prep compound pobj punct'),
 (213, 'compound ROOT prep compound pobj punct'),
 (213, 'compound ROOT prep compound pobj punct'),
 (213, 'compound ROOT prep compound pobj punct'),
 (214,
  'advcl prep nummod pobj punct conj cc conj punct ROOT prep nummod pobj punct conj dobj cc conj punct conj det amod nummod do

<IPython.core.display.Javascript object>

In [87]:
set((int(sentence[0]), dep_parser))

{263, 'ROOT cc conj punct'}

<IPython.core.display.Javascript object>

In [89]:
# Print out the result
print(
    "Total # of sentences:",
    len(combined_sentences_list),
    "\n# of Dependency Parser PICKED sentences:",
    len(set(dp_picked_sent)),
)

Total # of sentences: 52 
# of Dependency Parser PICKED sentences: 52


<IPython.core.display.Javascript object>

In [91]:
# Reform the sentence data

# Remove duplicates
dp_picked_a = set(list(dp_picked_sent))
dp_picked_new = list(dp_picked_a)

# Sort the order by ID
dp_picked_final = sorted(dp_picked_new)
dp_picked_final

[(212, 'compound compound ROOT prep dep dep amod pobj punct'),
 (213, 'compound ROOT prep compound pobj punct'),
 (214,
  'advcl prep nummod pobj punct conj cc conj punct ROOT prep nummod pobj punct conj dobj cc conj punct conj det amod nummod dobj mark det nsubj advcl acomp cc conj punct'),
 (215, 'ROOT advmod punct'),
 (216,
  'prep nummod compound nsubj punct compound conj punct conj punct conj cc conj ROOT prep nummod pobj punct'),
 (217, 'ROOT dobj punct conj cc conj punct conj nummod npadvmod advmod punct'),
 (218, 'ROOT dobj cc compound conj punct'),
 (219, 'ROOT amod compound dobj cc conj prep nummod punct pobj punct'),
 (220,
  'ROOT compound dobj punct appos punct amod conj punct conj cc compound conj punct appos nummod appos punct'),
 (221,
  'compound nsubj punct appos nummod npadvmod advmod cc conj aux relcl punct ROOT prep pobj mark advcl punct punct'),
 (222,
  'ROOT prep amod pobj punct conj prep pobj punct acl mark advcl cc conj advmod punct'),
 (223,
  'amod ROOT punc

<IPython.core.display.Javascript object>

In [94]:
for i in dp_picked_final:
    bb = list(i)

<IPython.core.display.Javascript object>

In [95]:
def convert(lst):
    return [i for item in lst for i in item.split()]

<IPython.core.display.Javascript object>

In [96]:
doc

Freeze and enjoy !

<IPython.core.display.Javascript object>

In [100]:
# Replace the dp parse with project tags

# tages = [("ROOT", "conj")]
# o_targe = []

dep_list = []
for i in dp_picked_final:
    p1 = i[1].split()
    p2 = map(
        lambda x: x
        if x == "ROOT" or x == "conj"  # or x == "nmod" or x == "acl" or x == "pcomp"
        else "O",
        p1,
    )
    new = map(lambda x: x if x == "O" else "U-Action", p2)
    dep_list.append((int(i[0]), list(new)))

dep_list

[(212, ['O', 'O', 'U-Action', 'O', 'O', 'O', 'O', 'O', 'O']),
 (213, ['O', 'U-Action', 'O', 'O', 'O', 'O']),
 (214,
  ['O',
   'O',
   'O',
   'O',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'O',
   'O',
   'O',
   'O',
   'U-Action',
   'O',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'U-Action',
   'O']),
 (215, ['U-Action', 'O', 'O']),
 (216,
  ['O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'U-Action',
   'O',
   'O',
   'O',
   'O']),
 (217,
  ['U-Action',
   'O',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'O',
   'U-Action',
   'O',
   'O',
   'O',
   'O']),
 (218, ['U-Action', 'O', 'O', 'O', 'U-Action', 'O']),
 (219, ['U-Action', 'O', 'O', 'O', 'O', 'U-Action', 'O', 'O', 'O', 'O', 'O']),
 (220,
  ['U-Action',
   'O',
   'O',
   'O',
   'O',
   'O',
   'O',
   'U-Action',
   'O'

<IPython.core.display.Javascript object>

In [137]:
# Assign modified tagged sentence to y_depend_output
y_depend_output = []
for i in dep_list:
    y_depend_output.append(i[1])

<IPython.core.display.Javascript object>

In [136]:
# Display the result of DP parse
print(
    metrics.flat_classification_report(
        y_test, y_depend_output, labels=["U-Action"], digits=3
    )
)

              precision    recall  f1-score   support

    U-Action      0.523     0.695     0.597        82

   micro avg      0.523     0.695     0.597        82
   macro avg      0.523     0.695     0.597        82
weighted avg      0.523     0.695     0.597        82



<IPython.core.display.Javascript object>

In [103]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Add butter to a saucepan and cover with lid")

displacy.render(
    doc, style="dep", jupyter=True,
)
print(doc.to_json())

{'text': 'Add butter to a saucepan and cover with lid', 'ents': [], 'sents': [{'start': 0, 'end': 43}], 'tokens': [{'id': 0, 'start': 0, 'end': 3, 'pos': 'VERB', 'tag': 'VB', 'dep': 'ROOT', 'head': 0}, {'id': 1, 'start': 4, 'end': 10, 'pos': 'NOUN', 'tag': 'NN', 'dep': 'dobj', 'head': 0}, {'id': 2, 'start': 11, 'end': 13, 'pos': 'ADP', 'tag': 'IN', 'dep': 'prep', 'head': 0}, {'id': 3, 'start': 14, 'end': 15, 'pos': 'DET', 'tag': 'DT', 'dep': 'det', 'head': 4}, {'id': 4, 'start': 16, 'end': 24, 'pos': 'NOUN', 'tag': 'NN', 'dep': 'pobj', 'head': 2}, {'id': 5, 'start': 25, 'end': 28, 'pos': 'CCONJ', 'tag': 'CC', 'dep': 'cc', 'head': 0}, {'id': 6, 'start': 29, 'end': 34, 'pos': 'VERB', 'tag': 'VB', 'dep': 'conj', 'head': 0}, {'id': 7, 'start': 35, 'end': 39, 'pos': 'ADP', 'tag': 'IN', 'dep': 'prep', 'head': 6}, {'id': 8, 'start': 40, 'end': 43, 'pos': 'NOUN', 'tag': 'NN', 'dep': 'pobj', 'head': 7}]}


<IPython.core.display.Javascript object>

In [114]:
##########################
# The end
##########################

<IPython.core.display.Javascript object>

In [197]:
data["tag"].value_counts()

O               2498
U-Action         366
U-Ingredient     241
L-Ingredient      70
B-Ingredient      70
B-Utensil         60
U-Utensil         59
L-Utensil         59
I-Utensil         15
I-Ingredient       3
B-Action           2
L-Action           2
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [198]:
data.head(10)

,sentence#,word,pos,tag
0,0.0,Preheat,VB,U-Action
1,0.0,oven,NN,U-Utensil
2,0.0,to,IN,O
3,0.0,425,CD,O
4,0.0,degrees,NNS,O
5,0.0,F.,NN,O
6,1.0,Press,NN,U-Action
7,1.0,dough,NN,O
8,1.0,into,IN,O
9,1.0,the,DT,O


<IPython.core.display.Javascript object>

In [117]:
data["tag"].value_counts()

O               2498
U-Action         366
U-Ingredient     241
L-Ingredient      70
B-Ingredient      70
B-Utensil         60
U-Utensil         59
L-Utensil         59
I-Utensil         15
I-Ingredient       3
B-Action           2
L-Action           2
Name: tag, dtype: int64

<IPython.core.display.Javascript object>

In [118]:
print(len(data["word"].values))

3445


<IPython.core.display.Javascript object>

In [119]:
words = list(set(data["word"].values))
len(words)

869

<IPython.core.display.Javascript object>

In [120]:
agg_func = lambda s: [
    (w, p, t)
    for w, p, t in zip(
        s["word"].values.tolist(), s["pos"].values.tolist(), s["tag"].values.tolist()
    )
]

<IPython.core.display.Javascript object>

In [121]:
grouped = data.groupby("sentence#").apply(agg_func)

<IPython.core.display.Javascript object>

In [122]:
sentences = [s for s in grouped]

<IPython.core.display.Javascript object>

In [123]:
len(sentences)

264

<IPython.core.display.Javascript object>

In [124]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        # set the bias for each word
        "bias": 1.0,
        # get the lower case form of the word
        "word.lower()": word.lower(),
        # get last 3 letters for the word
        "word[-3:]": word[-3:],
        # get last 2 letters for the word
        "word[-2:]": word[-2:],
        # check whether the word is uppercase or not
        "word.isupper()": word.isupper(),
        # check whether the word is title case or not
        "word.istitle()": word.istitle(),
        # check whether the word is digit or not, useful to identifying quantities which will be tagged as 'O'
        "word.isdigit()": word.isdigit(),
        # specifying the pos for word
        "postag": postag,
        # get first 2 letters for the POS tag
        "postag[:2]": postag[:2],
    }

    # if word is starting of sentence
    if i > 0:

        # if word is not the beginning of sentence
        # then get the word before it i.e. i-1 index
        word1 = sent[i - 1][0]

        # then get the pos before it i.e. i-1 index
        postag1 = sent[i - 1][1]

        features.update(
            {
                # setting the lower form of word at index i-1
                "-1:word.lower()": word1.lower(),
                # checking if the word at index i-1 is titlecase
                "-1:word.istitle()": word1.istitle(),
                # checking if the word at index i-1 is uppercase
                "-1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i-1
                "-1:postag": postag1,
                # get first 2 letters for the POS tag for i-1 indexed word
                "-1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the BOS or Begining of sentence to True
        features["BOS"] = True

    # if word is at the end of sentence
    if i < len(sent) - 1:

        # if word is not the end of sentence
        # then get the word after it i.e. i+1 index
        word1 = sent[i + 1][0]

        # then get the pos after it i.e. i+1 index
        postag1 = sent[i + 1][1]

        features.update(
            {
                # setting the lower form of word at index i+1
                "+1:word.lower()": word1.lower(),
                # checking if the word at index i+1 is titlecase
                "+1:word.istitle()": word1.istitle(),
                # checking if the word at index i+1 is titlecase
                "+1:word.isupper()": word1.isupper(),
                # setting the pos of word at index i+1
                "+1:postag": postag1,
                # get first 2 letters for the POS tag for i+1 indexed word
                "+1:postag[:2]": postag1[:2],
            }
        )
    else:
        # setting the EOS or End of sentence to True
        features["EOS"] = True

    return features

<IPython.core.display.Javascript object>

In [125]:
def sent2features(sent):
    """Convert sentences which are lists containing (w, p, t) into features"""
    return [word2features(sent, i) for i in range(len(sent))]

<IPython.core.display.Javascript object>

In [126]:
def sent2labels(sent):
    """Retrieve all the labels from sentences which are lists containing (w, p, t)"""
    return [label for token, postag, label in sent]

<IPython.core.display.Javascript object>

In [127]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

<IPython.core.display.Javascript object>

In [128]:
X

[[{'bias': 1.0,
   'word.lower()': 'preheat',
   'word[-3:]': 'eat',
   'word[-2:]': 'at',
   'word.isupper()': False,
   'word.istitle()': True,
   'word.isdigit()': False,
   'postag': 'VB',
   'postag[:2]': 'VB',
   'BOS': True,
   '+1:word.lower()': 'oven',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'NN',
   '+1:postag[:2]': 'NN'},
  {'bias': 1.0,
   'word.lower()': 'oven',
   'word[-3:]': 'ven',
   'word[-2:]': 'en',
   'word.isupper()': False,
   'word.istitle()': False,
   'word.isdigit()': False,
   'postag': 'NN',
   'postag[:2]': 'NN',
   '-1:word.lower()': 'preheat',
   '-1:word.istitle()': True,
   '-1:word.isupper()': False,
   '-1:postag': 'VB',
   '-1:postag[:2]': 'VB',
   '+1:word.lower()': 'to',
   '+1:word.istitle()': False,
   '+1:word.isupper()': False,
   '+1:postag': 'IN',
   '+1:postag[:2]': 'IN'},
  {'bias': 1.0,
   'word.lower()': 'to',
   'word[-3:]': 'to',
   'word[-2:]': 'to',
   'word.isupper()': False,
   'word.istitle()'

<IPython.core.display.Javascript object>

In [129]:
# split into train and test
boundary = math.ceil(len(X) * 0.8)

<IPython.core.display.Javascript object>

In [130]:
boundary

212

<IPython.core.display.Javascript object>

In [131]:
# train data
x_train = X[:boundary]
y_train = y[:boundary]

# test data
x_test = X[boundary:]
y_test = y[boundary:]

<IPython.core.display.Javascript object>

In [132]:
len(x_train)

212

<IPython.core.display.Javascript object>

In [133]:
print(len(x_train))
print(len(x_test))

212
52


<IPython.core.display.Javascript object>

In [134]:
print(sentences)

[[('Preheat', 'VB', 'U-Action'), ('oven', 'NN', 'U-Utensil'), ('to', 'IN', 'O'), ('425', 'CD', 'O'), ('degrees', 'NNS', 'O'), ('F.', 'NN', 'O')], [('Press', 'NN', 'U-Action'), ('dough', 'NN', 'O'), ('into', 'IN', 'O'), ('the', 'DT', 'O'), ('bottom', 'NN', 'O'), ('and', 'CC', 'O'), ('sides', 'NNS', 'O'), ('of', 'IN', 'O'), ('a', 'DT', 'O'), ('12', 'CD', 'O'), ('inch', 'NN', 'O'), ('pizza', 'NN', 'B-Utensil'), ('pan', 'NN', 'L-Utensil'), ('.', '.', 'O')], [('Bake', 'VB', 'U-Action'), ('for', 'IN', 'O'), ('5', 'CD', 'O'), ('minutes', 'NNS', 'O'), ('until', 'IN', 'O'), ('set', 'NN', 'O'), ('but', 'CC', 'O'), ('not', 'RB', 'O'), ('browned', 'VBN', 'U-Action'), ('.', '.', 'O')], [('Cut', 'VBN', 'U-Action'), ('sausage', 'NN', 'U-Ingredient'), ('into', 'IN', 'O'), ('small', 'JJ', 'O'), ('pieces', 'NNS', 'O'), ('.', '.', 'O')], [('Whisk', 'VB', 'U-Action'), ('eggs', 'NNS', 'U-Ingredient'), ('and', 'CC', 'O'), ('milk', 'NN', 'U-Ingredient'), ('in', 'IN', 'O'), ('a', 'DT', 'O'), ('bowl', 'NN', 'U

<IPython.core.display.Javascript object>

In [135]:
# creating CRF model with Gradient Descent
crf = CRF(
    algorithm="lbfgs",
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False,
)

<IPython.core.display.Javascript object>

In [ ]:
# fitting the model using train data
crf.fit(x_train, y_train)

In [ ]:
# to get all the labels/tags of data
labels = list(crf.classes_)

In [ ]:
labels

Since we are not interested in 'O' tags we will check the performance of the CRF model using f1 scores for every tag except O tags.

In [ ]:
labels.remove("O")
labels

In [ ]:
# performing predictions based on the fitted model
y_pred = crf.predict(x_test)

In [ ]:
# finding the f1 score
metrics.flat_f1_score(y_test, y_pred, average="weighted", labels=labels)

In [ ]:
# finding the f1 score
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

The model overfits!!

In [ ]:
len(data[((data["word"] == "Boil") | (data["word"] == "boil"))])

In [ ]:
len(data[((data["word"] == "Cover") | (data["word"] == "cover"))])

In [ ]:
len(data[((data["word"] == "Preheat") | (data["word"] == "preheat"))])

In [ ]:
eli5.show_weights(crf, top=30)

**Hyperparameter Optimization**

In [ ]:
# creating a CRF Hyperparameter tuned model
crf_hp = CRF(
    algorithm="lbfgs", c1=10, c2=0.1, max_iterations=100, all_possible_transitions=False
)

# parameters to hypertune
params_space = {"c1": scipy.stats.expon(scale=0.5), "c2": scipy.stats.expon(scale=0.05)}

# metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, average="weighted", labels=labels)

# search using 3 fold cross validation
rs = RandomizedSearchCV(
    crf_hp, params_space, cv=3, verbose=1, n_jobs=-1, n_iter=50, scoring=f1_scorer
)

In [ ]:
# fitting the hyperparameters
rs.fit(x_train, y_train)

Best Parameters:

In [ ]:
print("best params:", rs.best_params_)
print("best CV score:", rs.best_score_)

In [ ]:
# set the best estimator
crf_hp_tuned = rs.best_estimator_

In [ ]:
# predict using the best CRF model
y_pred = crf_hp_tuned.predict(x_test)

In [ ]:
# print the f1 evaluation metric
print(metrics.flat_classification_report(y_test, y_pred, labels=labels, digits=3))

In [ ]:
# transition features: given B-Utensil then what is score for I-Utensil

In [ ]:
# state features: what property of feature describes the tag

In [ ]:
eli5.show_weights(crf_hp_tuned, top=30)

## Untagged Data Stuff

In [ ]:
untagged_test_data = pd.read_csv("./Untagged Test Data/untagged_test_data.csv")

In [ ]:
untagged_test_data.head()

In [ ]:
len(untagged_test_data)

In [ ]:
# 30:5:30

In [ ]:
agg_func_test = lambda s: [
    (w, p) for w, p in zip(s["word"].values.tolist(), s["pos"].values.tolist())
]

In [ ]:
grouped_test = untagged_test_data.groupby(["recipe_name", "Step#"]).apply(agg_func_test)

In [ ]:
test_sentences = [s for s in grouped_test]

In [ ]:
len(test_sentences)

In [ ]:
X_untagged_test = [sent2features(s) for s in test_sentences]

In [ ]:
len(X_untagged_test)

In [ ]:
pred_test = crf_hp_tuned.predict(X_untagged_test)

In [ ]:
len(pred_test)

In [ ]:
count = 0
for l in pred_test:
    for tag in l:
        if tag == "U-Action":
            count += 1

In [ ]:
count

In [ ]:
len(list(untagged_test_data["word"].values))

In [ ]:
len(list(set(untagged_test_data["word"].values)))

Boostrap output to the untagged data df

In [ ]:
flat_list = []
for sublist in pred_test:
    for item in sublist:
        flat_list.append(item)

In [ ]:
len(flat_list)

In [ ]:
untagged_test_data["Predicted Output"] = flat_list

In [ ]:
len(untagged_test_data[untagged_test_data["Predicted Output"] == "U-Action"])

In [ ]:
untagged_test_data[untagged_test_data["Predicted Output"] == "U-Action"]